In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tensorflow.examples.tutorials.mnist import input_data


%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
location = 'E:/Dataset/MNIST/'
mnist = input_data.read_data_sets(location,one_hot=True)
print('\nDownload finished!')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:/Dataset/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:/Dataset/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting E:/Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting E:/Dataset/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Download finished!


In [3]:
#设定超参数
learning_rate = 0.01
training_iters = 5000
batch_size = 128
display_step = 10


n_inputs = 28  #列数目
n_steps = 28   #行数目
n_hidden_units = 128  #隐藏层神经元的个数是128个
n_classes = 10  #总共有10类

In [4]:
#Graph 输入
x = tf.placeholder(dtype=tf.float32,shape=[None,n_steps,n_inputs])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_classes])

In [5]:
Weights = {
    'in':  tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}

Biases  = {
    'in':  tf.Variable(tf.random_normal([n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [6]:
def RNN(X,weight,biase):
    #hidden layer for input
    #X --> 128 * 28行 * 28列 
    X = tf.reshape(X,[-1,n_inputs])
    X_in = tf.matmul(X,weight['in']) + biase['in']
    X_in = tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    #cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
    init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    output,states = tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=init_state,time_major=False)
    
    #hidden layer for output
    result = tf.matmul(states[1],Weights['out'])+biase['out']
    
    
    return result
pred = RNN(x,Weights,Biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))

train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)


correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy_caculate = tf.reduce_mean(tf.cast(correct_pred,tf.float32))


init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
with tf.Session() as sess:
    sess.run(init)
    steps = 0
    for steps  in range(training_iters):
        batch = mnist.train.next_batch(batch_size)
        batch_x = batch[0].reshape([-1,n_steps,n_inputs])
        batch_y = batch[1]
        
        sess.run(train_op,feed_dict = {x:batch_x, y:batch[1]})
        
        if steps%400 ==0:
            print('iterations:',steps ,' 准确率是：',sess.run(accuracy_caculate,feed_dict = { x:batch_x, y:batch_y}),\
                 '   损失： ',sess.run(cost,feed_dict = { x:batch_x, y:batch_y}))
        steps+=1

iterations: 0  准确率是： 0.0625    损失：  10.226744
iterations: 400  准确率是： 0.765625    损失：  0.69436026
iterations: 800  准确率是： 0.859375    损失：  0.44804823
iterations: 1200  准确率是： 0.9296875    损失：  0.22368234
iterations: 1600  准确率是： 0.9375    损失：  0.18138818
iterations: 2000  准确率是： 0.9453125    损失：  0.2163907
iterations: 2400  准确率是： 0.953125    损失：  0.24037042
iterations: 2800  准确率是： 0.9609375    损失：  0.14124542
iterations: 3200  准确率是： 0.96875    损失：  0.06935243
iterations: 3600  准确率是： 0.9921875    损失：  0.0348662
iterations: 4000  准确率是： 0.9765625    损失：  0.08486172
iterations: 4400  准确率是： 0.96875    损失：  0.09939143
iterations: 4800  准确率是： 0.953125    损失：  0.17768611
